In [ ]:
# import nest_asyncio
# nest_asyncio.apply()
# 
# import asyncio
# from slack import RTMClient
# 
# @RTMClient.run_on(event="message")
# def list_message(**payload):
#     data = payload['data']
#     print(data)  # Handle the incoming message data as per your requirements
# 
# with open('slack-api-token.txt', 'r') as file:
#     slack_token = file.read().strip()
# 
# rtm_client = RTMClient(token=slack_token)
# asyncio.ensure_future(rtm_client.start())
# asyncio.get_event_loop().run_forever()

/Users/jmcaffee/git/python-playground-v2/venv/lib/python3.9/site-packages/slack/deprecation.py:14: UserWarning: slack package is deprecated. Please use slack_sdk.web/webhook/rtm package instead. For more info, go to https://slack.dev/python-slack-sdk/v3-migration/
  warnings.warn(message)


In [1]:
# from slack_sdk.rtm_v2 import RTMClient
# with open('slack-api-token.txt', 'r') as file:
#     slack_token = file.read().strip()
# 
# rtm = RTMClient(token=slack_token)
# 
# @rtm.on("message")
# def handle(client: RTMClient, event: dict):
#     if 'Hello' in event['text']:
#         channel_id = event['channel']
#         thread_ts = event['ts']
#         user = event['user'] # This is not username but user ID (the format is either U*** or W***)
# 
#         client.web_client.chat_postMessage(
#             channel=channel_id,
#             text=f"Hi <@{user}>!",
#             thread_ts=thread_ts
#         )
# 
# rtm.connect()

In [2]:
from llama_cpp import Llama
max_tokens = 2048
# llama_llm = Llama(model_path="/Volumes/4Tera_SSD_2022/llm_models/llama-2-7b-chat.Q5_K_M.gguf", chat_format="llama-2")
llm = Llama(model_path="/Volumes/Macintosh HD/Users/jmcaffee/llm-models/llama-2-7b-chat.Q5_K_M.gguf", chat_format="llama-2", n_ctx=max_tokens)
llm.verbose = False

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Volumes/Macintosh HD/Users/jmcaffee/llm-models/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight 

In [4]:
import time
def prompt2(question):
    start_time_seconds = time.time()
    output = llm.create_chat_completion(
        max_tokens=max_tokens,
        messages = [
            {
                "role": "system",
                "content": "You are an assistant who perfectly responds to questions with as many details as possible, without making up facts."
            },
            {
                "role": "user",
                "content": question
            }
        ]
    )
    elapsed_time_seconds = time.time() - start_time_seconds
    # print(output)
    answer = output['choices'][0]['message']['content']
    print(f'answer generated in {elapsed_time_seconds} seconds')
    print(f"Q: {question} \nA:{answer}")
    return answer

In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
from slack_sdk.rtm_v2 import RTMClient

# Load your Slack API token
with open('slack-api-token.txt', 'r') as file:
    slack_token = file.read().strip()

# Initialize the RTMClient
rtm = RTMClient(token=slack_token)

# Define the event handler
@rtm.on("message")
def handle(client: RTMClient, event: dict):
    print(f"event: {event}")
    message_received = event.get('text', '')
    channel_id = event['channel']
    thread_ts = event['ts']
    user = event['user']  # User ID

    
    # Post a message to the channel
    client.web_client.chat_postMessage(
        channel=channel_id,
        text=f"Having llm answer the question: {message_received}",
        thread_ts=thread_ts
    )
    
    answer = prompt2(message_received)
    client.web_client.chat_postMessage(
        channel=channel_id,
        text=f"Answer: {answer}",
        thread_ts=thread_ts
    )
# Running the event loop
asyncio.ensure_future(rtm.start())
asyncio.get_event_loop().run_forever()


event: {'type': 'message', 'channel': 'D06ATFU9MAT', 'text': 'Hello', 'blocks': [{'type': 'rich_text', 'block_id': 'HnjxV', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'Hello'}]}]}], 'user': 'U02EJ467WLX', 'client_msg_id': 'f5626b6d-8305-4041-84ac-05226e9c738f', 'team': 'TB14JBH0C', 'source_team': 'E01BQQJKZQS', 'user_team': 'E01BQQJKZQS', 'suppress_notification': False, 'event_ts': '1703183272.832849', 'ts': '1703183272.832849'}
event: {'type': 'message', 'subtype': 'message_replied', 'message': {'client_msg_id': 'f5626b6d-8305-4041-84ac-05226e9c738f', 'type': 'message', 'text': 'Hello', 'user': 'U02EJ467WLX', 'ts': '1703183272.832849', 'blocks': [{'type': 'rich_text', 'block_id': 'HnjxV', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'Hello'}]}]}], 'team': 'TB14JBH0C', 'thread_ts': '1703183272.832849', 'reply_count': 1, 'reply_users_count': 1, 'latest_reply': '1703183273.302039', 'reply_users': ['B06BLH6F9S5'], '

KeyboardInterrupt: 

In [2]:
!pip install -U sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 2.8 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 5.2 MB/s eta 0:00:00
  Using cached pathos-0.3.1-py3-none-any.whl (82 kB)
  Using cached tblib-1.7.0-py2.py3-none-any.whl (12 kB)
  Using cached schema-0.7.5-py2.py3-none-any.whl (17 kB)
  Using cached importlib_metadata-6.11.0-py3-none-any.whl (23 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached smdebug_rulesconfig-1.0.1-py2.py3-none-any.whl (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 13.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 9.8 MB/s eta 0:00:00:

In [1]:
import io
class LineIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord('\n'):
                self.read_pos += len(line)
                return line[:-1]
            try:
                chunk = next(self.byte_iterator)
            except StopIteration:
                if self.read_pos < self.buffer.getbuffer().nbytes:
                    continue
                raise
            if 'PayloadPart' not in chunk:
                print('Unknown event type:' + chunk)
                continue
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk['PayloadPart']['Bytes'])

In [3]:
# stream https://aws.amazon.com/blogs/machine-learning/elevating-the-generative-ai-experience-introducing-streaming-support-in-amazon-sagemaker-hosting/
# run !gimme-aws-creds, then aws_switch sofi-bedrock-bdp-production
import boto3
import io
import json 

# session = boto3.Session(profile_name='AssumeBedrockPOC')
session = boto3.Session(profile_name='sofi-bedrock-bdp-production')
smr = session.client('sagemaker-runtime', 'us-west-2')
#AssumeBedrockPOC

def sagemaker_prompt(question, on_text_received):
    print(f"sagemaker question: {question}")
    system_message = "You are a helpful assistant that does not use superfluous pleasantries. Avoiding ending your reply with questions.  If a question does not make sense, call out why it doesn't make sense, and don't attempt to answer. If you don't know the answer to a question, do not make up an answer."
    prompt=f'''[INST] <<SYS>>
        {system_message}
        <</SYS>>
        {question} [/INST]'''
    
    # hyperparameters for llm
    payload = {
      "inputs": prompt,
      "parameters": {
        "do_sample": True,
        "top_p": 0.7,
        "temperature": 0.7,
        "top_k": 10,
        "max_new_tokens": 500,
        "repetition_penalty": 1.03,
        "stop": ["<|endoftext|>"]
      },
      "stream": True  
    }
    stop_token = "<|endoftext|>"
    # llm.deserializer=StreamDeserializer()
    endpoint_name = "Llama-2-13B-Chat-fp16-v7-2023-12-21-17-51-07-902"
    resp = smr.invoke_endpoint_with_response_stream(EndpointName=endpoint_name, Body=json.dumps(payload), ContentType='application/json')
    event_stream = resp['Body']
    start_json = b'{'
    for line in LineIterator(event_stream):
        if line != b'' and start_json in line:
            data = json.loads(line[line.find(start_json):].decode('utf-8'))
            if data['token']['text'] != stop_token:
                on_text_received(data['token']['text'])
                # print(data['token']['text'],end='')

def handle_text_received(text):
    print(text, end='')

sagemaker_prompt("What is a camel?", handle_text_received)                

In [6]:
# !gimme-aws-creds
# !aws_switch sofi-bedrock-bdp-production

zsh:1: command not found: aws_switch


event: {'type': 'message', 'subtype': 'channel_join', 'channel': 'C06BXU1T6P2', 'text': '<@U02EJ467WLX> has joined the channel', 'user': 'U02EJ467WLX', 'team': 'TB14JBH0C', 'event_ts': '1703197636.366049', 'ts': '1703197636.366049'}event: {'type': 'message', 'channel': 'C06BXU1T6P2', 'text': '<@U06B7U1478S> what is a nucleous?', 'blocks': [{'type': 'rich_text', 'block_id': 'JHt8m', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'user', 'user_id': 'U06B7U1478S'}, {'type': 'text', 'text': ' what is a nucleous?'}]}]}], 'user': 'U02EJ467WLX', 'client_msg_id': '20a19d97-6fe7-4c35-ad14-12c4699f0729', 'team': 'TB14JBH0C', 'source_team': 'E01BQQJKZQS', 'user_team': 'E01BQQJKZQS', 'suppress_notification': False, 'event_ts': '1703197661.088719', 'ts': '1703197661.088719'}
event: {'type': 'message', 'subtype': 'channel_join', 'channel': 'C06BXU1T6P2', 'text': '<@U06B7U1478S> has joined the channel', 'user': 'U06B7U1478S', 'team': 'TB14JBH0C', 'inviter': 'U02EJ467WLX', 'event_ts'

KeyboardInterrupt: 

# Begin new attempt using threading 

In [4]:
import asyncio
import nest_asyncio
import threading
nest_asyncio.apply()
from slack_sdk.rtm_v2 import RTMClient

# Load your Slack API token
with open('slack-api-token.txt', 'r') as file:
    slack_token = file.read().strip()

# Initialize the RTMClient
rtm = RTMClient(token=slack_token)

# Define the event handler
@rtm.on("message")
def handle(client: RTMClient, event: dict):
    # print(f"event: {event}")
    message_received = event.get('text', '')
    # channel_id = event['channel']
    # thread_ts = event['ts']
    # user = event['user']  # User ID
    ai_bot_user = "<@U06B7U1478S>"
    
    # Post a message to the channel
    # Check if the message is addressed to @ai
    if message_received.startswith(ai_bot_user):
        print('ai_bot_user')
        
        def handle_ai_bot_message(client: RTMClient, event: dict):
            # print(f'handle ai bot message: {event}')
            message_received = event.get('text', '')
            channel_id = event['channel']
            thread_ts = event['ts']
            
            prompt = message_received.replace(ai_bot_user, '')
            cumulative_text = f"{prompt}\n"
            print(cumulative_text)
            # Initial message sent to the channel
            initial_response = client.web_client.chat_postMessage(
                channel=channel_id,
                text=cumulative_text,
                thread_ts=thread_ts
            )
    
            # Retrieve timestamp of the initial response
            initial_response_ts = initial_response['ts']
            last_update_length = len(cumulative_text)
            
            def handle_text_received(text):
                nonlocal cumulative_text, last_update_length
                cumulative_text += text
                # print(text, end='')
                # Edit the existing message with the new text
                if len(cumulative_text) - last_update_length >= 20 or "</s>" in cumulative_text:
                    client.web_client.chat_update(
                        channel=channel_id,
                        ts=initial_response_ts,  # Timestamp of the message to update
                        text=cumulative_text,
                        thread_ts=thread_ts
                    )
                    last_update_length = len(cumulative_text)
    
            sagemaker_prompt(message_received, handle_text_received)

        thread = threading.Thread(
            target=handle_ai_bot_message,
            args=(client, event)
        )
        thread.start()
        # handle_ai_bot_message(client, event)
    
# Running the event loop
# asyncio.ensure_future(rtm.start())
# asyncio.get_event_loop().run_forever()

loop = asyncio.get_event_loop()
try:
    asyncio.ensure_future(rtm.start())
    loop.run_forever()
except KeyboardInterrupt:
    # Handle the interrupt gracefully
    print("Interrupted by user, shutting down.")
finally:
    # Perform any cleanup here if necessary
    loop.stop()
    # loop.close()

ai_bot_user
 why is the moon silver?
sagemaker question: <@U06B7U1478S> why is the moon silver?
ai_bot_user
 how many shark attacks are there per year on average?

sagemaker question: <@U06B7U1478S> how many shark attacks are there per year on average?
ai_bot_user
 write some python code to find prime numbers
sagemaker question: <@U06B7U1478S> write some python code to find prime numbers
ai_bot_user
 tell me a story about a Buddhist monk
sagemaker question: <@U06B7U1478S> tell me a story about a Buddhist monk
ai_bot_user
 how does fire work?

sagemaker question: <@U06B7U1478S> how does fire work?
ai_bot_user
 how do I obtain enlightenment?

sagemaker question: <@U06B7U1478S> how do I obtain enlightenment?
ai_bot_user
 what is the likelihood of life beyond death?

sagemaker question: <@U06B7U1478S> what is the likelihood of life beyond death?
ai_bot_user
 what are some states of consciousness?

sagemaker question: <@U06B7U1478S> what are some states of consciousness?
ai_bot_user
 how do

In [ ]:
# # sagemaker rtm
# import asyncio
# import nest_asyncio
# nest_asyncio.apply()
# from slack_sdk.rtm_v2 import RTMClient
# 
# # Load your Slack API token
# with open('slack-api-token.txt', 'r') as file:
#     slack_token = file.read().strip()
# 
# # Initialize the RTMClient
# rtm = RTMClient(token=slack_token)
# 
# # Define the event handler
# @rtm.on("message")
# def handle(client: RTMClient, event: dict):
#     print(f"event: {event}")
#     message_received = event.get('text', '')
#     channel_id = event['channel']
#     thread_ts = event['ts']
#     user = event['user']  # User ID
#     ai_bot_user = "<@U06B7U1478S>"
#     
#     # Post a message to the channel
#     # Check if the message is addressed to @ai
#     if ai_bot_user in message_received:
#         prompt = message_received.replace(ai_bot_user, '')
#         cumulative_text = f"Sending prompt to LLM: {prompt}"
#         # Initial message sent to the channel
#         initial_response = client.web_client.chat_postMessage(
#             channel=channel_id,
#             text=cumulative_text,
#             thread_ts=thread_ts
#         )
# 
#         # Retrieve timestamp of the initial response
#         initial_response_ts = initial_response['ts']
# 
#         def handle_text_received(text):
#             global cumulative_text
#             cumulative_text += text
#             print(text, end='')
#             # Edit the existing message with the new text
#             client.web_client.chat_update(
#                 channel=channel_id,
#                 ts=initial_response_ts,  # Timestamp of the message to update
#                 text=cumulative_text,
#                 thread_ts=thread_ts
#             )
# 
#         sagemaker_prompt(message_received, handle_text_received)
#     
# # Running the event loop
# asyncio.ensure_future(rtm.start())
# asyncio.get_event_loop().run_forever()


In [5]:
import re
message = "/summarize-url https://api.slack.com/legacy/message-menus"
pattern = r'^(\/summarize-url)\s+(https?:\/\/[^\s]+)(.*)$'
match = re.match(pattern, message)
print(match)

slash_command, url, new_message_with_command_stripped = match.groups()
print(url)

None


AttributeError: 'NoneType' object has no attribute 'groups'